In [2]:
import pandas as pd

In [3]:
data = pd.read_excel("hotel_reviews_with_score_final.xlsx")

data.head()

,no,Hotel name,rating,user type,room type,nights,review,Sentiment,score_normalized,sentiment_score,Score_Final
0,2,فندق 72,2,مسافر منفرد,غرفة ديلوكس مزدوجة أو توأم,أقمت ليلة واحدة,“ممتاز”. النظافة والطاقم متعاون.,Negative,0.4,-1,-2.3
1,3,فندق 72,5,زوج,غرفة ديلوكس مزدوجة أو توأم,أقمت ليلة واحدة,استثنائي. سهولة إنهاء المعاملة في الاستقبال. ل...,Positive,1.0,1,3.0
2,16,فندق 72,5,زوج,-,أقمت ليلتين,استثنائي. انصح بأختيار الاسويت و بالاخص غرفه ر...,Positive,1.0,1,3.0
3,20,فندق 72,1,زوج,غرفة قياسية مزدوجة,أقمت ليلة واحدة,“استغرب تقييم الفندق كخمس نجوم”. لا شي. يستحق ...,Negative,0.2,-1,-2.4
4,23,فندق 72,4,زوج,غرفة ديلوكس مزدوجة أو توأم,أقمت ليلتين,جيد. المكان جميل وهاديء. كل شي جيد ونظيف بس كا...,Positive,0.8,1,2.9


In [4]:
# Créer des IDs pour utilisateurs et hôtels
user_ids = data["user type"].astype("category").cat.codes
hotel_ids = data["Hotel name"].astype("category").cat.codes

num_users = user_ids.nunique()
num_hotels = hotel_ids.nunique()

print("Nombre d'utilisateurs:", num_users)
print("Nombre d'hôtels:", num_hotels)


Nombre d'utilisateurs: 6
Nombre d'hôtels: 1173


In [5]:
import torch

# Source = utilisateurs
edge_src = torch.tensor(user_ids.values, dtype=torch.long)

# Destination = hôtels (offset)
edge_dst = torch.tensor(hotel_ids.values + num_users, dtype=torch.long)

# Edge index (format PyG)
edge_index = torch.stack([edge_src, edge_dst], dim=0)

# Poids des arêtes = Score Final
edge_weight = torch.tensor(data["Score_Final"].values, dtype=torch.float)

print("Edge index shape:", edge_index.shape)
print("Edge weight shape:", edge_weight.shape)


Edge index shape: torch.Size([2, 105698])
Edge weight shape: torch.Size([105698])


In [6]:
num_nodes = num_users + num_hotels

x = torch.ones((num_nodes, 1))  # feature simple

print("Nombre total de nœuds:", num_nodes)
print("Shape des features:", x.shape)


Nombre total de nœuds: 1179
Shape des features: torch.Size([1179, 1])


In [7]:
from torch_geometric.data import Data

graph_data = Data(
    x=x,
    edge_index=edge_index,
    edge_weight=edge_weight
)

print(graph_data)


Data(x=[1179, 1], edge_index=[2, 105698], edge_weight=[105698])
